## مقدمه

در این درس با موارد زیر آشنا می‌شوید:
- تابع فراخوانی چیست و چه کاربردهایی دارد
- چطور با استفاده از Azure OpenAI یک فراخوانی تابع بسازیم
- چگونه یک فراخوانی تابع را در یک برنامه ادغام کنیم

## اهداف یادگیری

پس از پایان این درس، می‌دانید که چگونه:
- هدف استفاده از فراخوانی تابع را درک کنید
- فراخوانی تابع را با سرویس Azure Open AI راه‌اندازی کنید
- برای کاربرد برنامه خود، فراخوانی‌های تابع مؤثر طراحی کنید


## درک فراخوانی توابع

در این درس، می‌خواهیم یک قابلیت برای استارتاپ آموزشی خود بسازیم که به کاربران اجازه می‌دهد با استفاده از یک چت‌بات، دوره‌های فنی را پیدا کنند. ما دوره‌هایی را پیشنهاد می‌دهیم که با سطح مهارت، نقش فعلی و تکنولوژی مورد علاقه آن‌ها مطابقت داشته باشد.

برای انجام این کار، از ترکیبی از موارد زیر استفاده می‌کنیم:
 - `Azure Open AI` برای ایجاد یک تجربه چت برای کاربر
 - `Microsoft Learn Catalog API` برای کمک به کاربران در پیدا کردن دوره‌ها بر اساس درخواست آن‌ها
 - `Function Calling` برای گرفتن پرسش کاربر و ارسال آن به یک تابع جهت انجام درخواست به API

برای شروع، بیایید ببینیم اصلاً چرا باید از فراخوانی توابع استفاده کنیم:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # دریافت یک پاسخ جدید از GPT که می‌تواند پاسخ تابع را ببیند


print(second_response.choices[0].message)


### چرا فراخوانی تابع

اگر هر درس دیگری از این دوره را گذرانده باشید، احتمالاً با قدرت استفاده از مدل‌های زبانی بزرگ (LLM) آشنا شده‌اید. امیدوارم همچنین برخی از محدودیت‌های آن‌ها را هم دیده باشید.

فراخوانی تابع قابلیتی در سرویس Azure Open AI است که برای رفع محدودیت‌های زیر ارائه شده است:
1) فرمت پاسخ‌دهی یکسان و منظم
2) امکان استفاده از داده‌های منابع دیگر یک برنامه در یک گفت‌وگو

قبل از فراخوانی تابع، پاسخ‌های یک LLM ساختار مشخص و ثابتی نداشتند. توسعه‌دهندگان مجبور بودند کدهای اعتبارسنجی پیچیده‌ای بنویسند تا بتوانند هر نوع پاسخ را مدیریت کنند.

کاربران نمی‌توانستند به سوالاتی مثل «الان هوای استکهلم چطور است؟» پاسخ بگیرند. دلیلش این بود که مدل‌ها فقط تا زمانی که داده‌ها آموزش دیده بودند، اطلاعات داشتند.

بیایید به مثال زیر که این مشکل را نشان می‌دهد نگاه کنیم:

فرض کنید می‌خواهیم یک پایگاه داده از اطلاعات دانش‌آموزان بسازیم تا بتوانیم دوره مناسب را به آن‌ها پیشنهاد دهیم. در زیر دو توصیف از دانش‌آموزان داریم که از نظر داده‌های موجود بسیار شبیه به هم هستند.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

ما می‌خواهیم این را برای یک مدل زبانی ارسال کنیم تا داده‌ها را تجزیه کند. بعداً می‌توانیم از این داده‌ها در برنامه‌مان استفاده کنیم تا آن را به یک API ارسال کنیم یا در پایگاه داده ذخیره کنیم.

بیایید دو درخواست مشابه بسازیم که به مدل زبانی توضیح می‌دهد چه اطلاعاتی برای ما مهم است:


ما می‌خواهیم این را به یک مدل زبانی بزرگ ارسال کنیم تا بخش‌هایی که برای محصول ما مهم هستند را تجزیه و تحلیل کند. بنابراین می‌توانیم دو درخواست یکسان ایجاد کنیم تا به مدل زبانی بزرگ دستور دهیم:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


پس از ایجاد این دو پرامپت، آن‌ها را با استفاده از `openai.ChatCompletion` به LLM ارسال می‌کنیم. پرامپت را در متغیر `messages` ذخیره می‌کنیم و نقش را به `user` اختصاص می‌دهیم. این کار برای شبیه‌سازی پیامی از طرف کاربر است که به یک چت‌بات نوشته می‌شود.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

اکنون می‌توانیم هر دو درخواست را به مدل زبان بزرگ ارسال کنیم و پاسخ دریافتی را بررسی کنیم.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

حتی اگر دستورات یکسان باشند و توضیحات مشابهی داشته باشند، ممکن است فرمت‌های متفاوتی از ویژگی `Grades` دریافت کنیم.

اگر سلول بالا را چند بار اجرا کنید، فرمت می‌تواند به صورت `3.7` یا `3.7 GPA` باشد.

دلیل این موضوع این است که مدل زبانی داده‌های بدون ساختار را به صورت متن دریافت می‌کند و خروجی هم به صورت داده‌های بدون ساختار است. ما نیاز داریم که یک فرمت ساختاریافته داشته باشیم تا بدانیم هنگام ذخیره یا استفاده از این داده‌ها چه انتظاری باید داشته باشیم.

با استفاده از فراخوانی تابعی (functional calling)، می‌توانیم مطمئن شویم که داده‌های ساختاریافته دریافت می‌کنیم. هنگام استفاده از فراخوانی تابعی، مدل زبانی در واقع هیچ تابعی را اجرا یا فراخوانی نمی‌کند. بلکه ما یک ساختار برای پاسخ‌های مدل تعریف می‌کنیم تا مدل طبق آن پاسخ دهد. سپس از این پاسخ‌های ساختاریافته استفاده می‌کنیم تا بدانیم در برنامه‌هایمان باید کدام تابع را اجرا کنیم.


![نمودار جریان فراخوانی تابع](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.fa.png)


### موارد استفاده از فراخوانی توابع

**فراخوانی ابزارهای خارجی**  
چت‌بات‌ها در پاسخ دادن به سوالات کاربران بسیار خوب عمل می‌کنند. با استفاده از فراخوانی توابع، چت‌بات‌ها می‌توانند از پیام‌های کاربران برای انجام برخی کارها استفاده کنند. برای مثال، یک دانش‌آموز می‌تواند از چت‌بات بخواهد: «یک ایمیل به استاد من بفرست و بگو که من به کمک بیشتری در این موضوع نیاز دارم». این درخواست می‌تواند تابعی مثل `send_email(to: string, body: string)` را فراخوانی کند.

**ایجاد کوئری‌های API یا پایگاه داده**  
کاربران می‌توانند با زبان طبیعی اطلاعات مورد نیاز خود را پیدا کنند که این درخواست به یک کوئری یا درخواست API فرمت‌شده تبدیل می‌شود. برای مثال، یک معلم می‌پرسد: «کدام دانش‌آموزان آخرین تکلیف را انجام داده‌اند؟» که می‌تواند تابعی به نام `get_completed(student_name: string, assignment: int, current_status: string)` را فراخوانی کند.

**ایجاد داده‌های ساختاریافته**  
کاربران می‌توانند یک بلوک متن یا فایل CSV را وارد کنند و با کمک مدل زبانی بزرگ، اطلاعات مهم را از آن استخراج کنند. برای مثال، یک دانش‌آموز می‌تواند یک مقاله ویکی‌پدیا درباره توافق‌نامه‌های صلح را به فلش‌کارت‌های هوش مصنوعی تبدیل کند. این کار با استفاده از تابعی به نام `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` انجام می‌شود.


## ۲. ساخت اولین فراخوانی تابع

فرآیند ساخت یک فراخوانی تابع شامل ۳ مرحله اصلی است:
۱. فراخوانی API تکمیل چت با لیستی از توابع خود و یک پیام کاربر
۲. خواندن پاسخ مدل برای انجام یک عمل، مثلاً اجرای یک تابع یا فراخوانی API
۳. انجام یک فراخوانی دیگر به API تکمیل چت با پاسخ دریافتی از تابع خود تا از آن اطلاعات برای ساخت پاسخ به کاربر استفاده کنید.


![جریان یک فراخوانی تابع](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.fa.png)


### اجزای یک فراخوانی تابع

#### ورودی کاربر

اولین قدم ایجاد یک پیام کاربر است. این پیام می‌تواند به صورت پویا از مقدار ورودی یک فیلد متنی گرفته شود یا می‌توانید مستقیماً مقداری به آن اختصاص دهید. اگر اولین بار است که با API تکمیل چت کار می‌کنید، باید `role` و `content` پیام را مشخص کنیم.

`role` می‌تواند یکی از این سه مقدار باشد: `system` (ایجاد قوانین)، `assistant` (مدل) یا `user` (کاربر نهایی). برای فراخوانی تابع، این مقدار را به عنوان `user` و یک سؤال نمونه قرار می‌دهیم.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### ساختن توابع

در ادامه یک تابع و پارامترهای آن را تعریف می‌کنیم. در اینجا فقط از یک تابع به نام `search_courses` استفاده می‌کنیم اما شما می‌توانید چندین تابع بسازید.

**مهم**: توابع در پیام سیستمی به LLM اضافه می‌شوند و در میزان توکن‌های قابل استفاده شما محاسبه خواهند شد.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**تعاریف**

`name` - نام تابعی که می‌خواهیم فراخوانی شود.

`description` - این توضیحی است درباره نحوه عملکرد تابع. در اینجا مهم است که توضیح دقیق و واضح باشد.

`parameters` - فهرستی از مقادیر و قالبی که می‌خواهید مدل در پاسخ خود تولید کند.

`type` - نوع داده‌ای که ویژگی‌ها در آن ذخیره خواهند شد.

`properties` - فهرستی از مقادیر مشخصی که مدل برای پاسخ خود استفاده خواهد کرد.

`name` - نام ویژگی که مدل در پاسخ قالب‌بندی‌شده خود استفاده می‌کند.

`type` - نوع داده‌ای این ویژگی.

`description` - توضیح درباره ویژگی خاص.

**اختیاری**

`required` - ویژگی مورد نیاز برای اینکه فراخوانی تابع کامل شود.


### فراخوانی تابع
بعد از تعریف یک تابع، حالا باید آن را در فراخوانی به API تکمیل چت قرار دهیم. این کار را با اضافه کردن `functions` به درخواست انجام می‌دهیم. در این مثال، `functions=functions` است.

همچنین گزینه‌ای وجود دارد که مقدار `function_call` را روی `auto` قرار دهید. این یعنی اجازه می‌دهیم مدل زبانی تصمیم بگیرد که کدام تابع باید بر اساس پیام کاربر فراخوانی شود، به جای اینکه خودمان آن را تعیین کنیم.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

حالا بیایید به پاسخ نگاه کنیم و ببینیم چطور قالب‌بندی شده است:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

می‌بینید که نام تابع فراخوانی شده و مدل زبانی از پیام کاربر توانسته داده‌ها را برای قرار دادن در آرگومان‌های تابع پیدا کند.


## ۳. ادغام فراخوانی توابع در یک برنامه

بعد از اینکه پاسخ قالب‌بندی‌شده از LLM را تست کردیم، حالا می‌توانیم آن را در یک برنامه ادغام کنیم.

### مدیریت جریان

برای ادغام این موضوع در برنامه‌مان، بیایید مراحل زیر را دنبال کنیم:

ابتدا، فراخوانی سرویس‌های Open AI را انجام می‌دهیم و پیام را در متغیری به نام `response_message` ذخیره می‌کنیم.


In [ ]:
response_message = response.choices[0].message

اکنون تابعی را تعریف خواهیم کرد که API مایکروسافت لرن را فراخوانی می‌کند تا فهرستی از دوره‌ها را دریافت کند:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



به عنوان یک روش پیشنهادی، ابتدا بررسی می‌کنیم که آیا مدل می‌خواهد تابعی را فراخوانی کند یا نه. پس از آن، یکی از توابع موجود را ایجاد می‌کنیم و آن را با تابعی که فراخوانی شده مطابقت می‌دهیم.
سپس آرگومان‌های تابع را می‌گیریم و آن‌ها را به آرگومان‌هایی که از LLM آمده‌اند، نگاشت می‌کنیم.

در نهایت، پیام فراخوانی تابع و مقادیری که توسط پیام `search_courses` بازگردانده شده‌اند را اضافه می‌کنیم. این کار تمام اطلاعات لازم را در اختیار LLM قرار می‌دهد تا بتواند با زبان طبیعی به کاربر پاسخ دهد.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## چالش کدنویسی

کارت عالی بود! برای ادامه یادگیری درباره Azure Open AI Function Calling می‌تونی این پروژه رو بسازی: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
- پارامترهای بیشتری به تابع اضافه کن تا به یادگیرنده‌ها کمک کنه دوره‌های بیشتری پیدا کنن. می‌تونی پارامترهای موجود API رو اینجا ببینی:  
- یک فراخوانی تابع دیگه بساز که اطلاعات بیشتری مثل زبان مادری یادگیرنده رو دریافت کنه  
- مدیریت خطا اضافه کن تا اگر فراخوانی تابع یا API هیچ دوره مناسبی برنگردوند، پیام مناسبی نمایش داده بشه



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما برای دقت تلاش می‌کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطا یا نادقتی باشند. نسخه اصلی سند به زبان مادری آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، ترجمه حرفه‌ای توسط انسان توصیه می‌شود. ما هیچ مسئولیتی در قبال سوءتفاهم یا تفسیر نادرست ناشی از استفاده از این ترجمه نداریم.
